## MSDS 7330 



##  2018 MLB Pitchers Data Creation


In order to Get Started the MYSQL Databased must be created. This Jupyter notebook uses python 3, so if you don't have python 3 installed on your machine you might run into issues, as this was not tested on earlier versions. After Following the steps of this notebook you should be able to have all the data set up in your local database.

Make sure the following Packages are installed on your machine. If you are missing one you can install it by runnint either ```pip install <package>``` or ``` conda install <package>``` if you are using conda package manager.

In [1]:
import pandas as pd
import xlrd
import sqlalchemy
import mysql.connector

Next, import that pitcher data from the `data` directory

In [2]:
df = pd.read_excel("data/2018_pitching.xlsx", "mlb_stats_2018")

Get the shape of the data.

In [3]:
df.shape

(878, 36)

Run the next line in order to check if there is missing values in the dataframe.

In [4]:
df.isnull().values.any()

True

We know data is missing so lets count the missing items in each column.

In [5]:
df.isnull().sum()

Rk               0
Name             0
2018_salary    357
Age              0
Tm               0
Lg               0
W                0
L                0
W-L%           217
ERA              0
G                0
GS               0
GF               0
CG               0
SHO              0
SV               0
IP               0
H                0
R                0
ER               0
HR               0
BB               0
IBB              0
SO               0
HBP              0
BK               0
WP               0
BF               0
ERA+            53
FIP              2
WHIP             2
H9               2
HR9              2
BB9              2
SO9              2
SO/W            56
dtype: int64

We remove the records with missing values from the column.

In [6]:
df = df.dropna()

In [7]:
df.isnull().sum()

Rk             0
Name           0
2018_salary    0
Age            0
Tm             0
Lg             0
W              0
L              0
W-L%           0
ERA            0
G              0
GS             0
GF             0
CG             0
SHO            0
SV             0
IP             0
H              0
R              0
ER             0
HR             0
BB             0
IBB            0
SO             0
HBP            0
BK             0
WP             0
BF             0
ERA+           0
FIP            0
WHIP           0
H9             0
HR9            0
BB9            0
SO9            0
SO/W           0
dtype: int64

All missing values have been dropped from the dataset.

In [8]:
df.shape

(446, 36)

In [9]:
df.describe()

,Rk,2018_salary,Age,W,L,W-L%,ERA,G,GS,GF,...,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W
count,446.000000,4.460000e+02,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,...,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.000000,446.00000,446.000000
mean,439.887892,4.346355e+06,29.076233,3.387892,3.408072,0.479379,4.587937,28.143498,6.672646,6.825112,...,2.302691,253.105381,117.210762,4.333341,1.382312,8.967265,1.251345,3.473318,8.64148,3.081457
std,250.478277,5.842750e+06,3.591941,3.203893,2.769712,0.283506,3.137819,15.191868,9.116993,9.289157,...,2.393876,171.365947,70.856211,1.628891,0.446795,3.251434,0.938148,1.633905,2.19327,1.877630
min,4.000000,5.450000e+05,21.000000,0.000000,0.000000,0.000000,0.540000,1.000000,0.000000,0.000000,...,0.000000,11.000000,13.000000,1.230000,0.536000,3.700000,0.000000,0.500000,2.20000,0.290000
25%,229.250000,5.584250e+05,27.000000,1.000000,1.000000,0.333000,3.172500,18.000000,0.000000,0.000000,...,1.000000,129.250000,83.000000,3.390000,1.162250,7.500000,0.700000,2.400000,7.20000,1.947500
50%,430.000000,1.912500e+06,28.500000,2.000000,3.000000,0.500000,4.140000,24.000000,0.000000,4.000000,...,2.000000,199.000000,101.000000,4.120000,1.312500,8.600000,1.100000,3.200000,8.55000,2.710000
75%,655.500000,5.625000e+06,31.750000,5.000000,5.000000,0.667000,5.052500,41.750000,15.000000,9.000000,...,3.000000,364.750000,132.000000,4.970000,1.500000,9.700000,1.500000,4.100000,10.07500,3.597500
max,878.000000,3.400000e+07,41.000000,15.000000,14.000000,1.000000,45.000000,59.000000,25.000000,52.000000,...,15.000000,680.000000,825.000000,17.150000,7.000000,54.000000,9.000000,11.400000,16.20000,16.330000


Now we are ready to begin the process of creating our Database and table. First we need to make sure that we are clearing any running pool managers.

In [10]:
sqlalchemy.pool.clear_managers()

** IMPORTANT ** If you have a different `username` and `password` you need to make sure you change the next cell to match your credentials in order for this to work. The next cell creates a dictionary which is used to access the local MYSQL instance.

In [11]:
creds = dict(username = 'root',password = '', ip = '127.0.0.1')

In [12]:
creds

{'username': 'root', 'password': '', 'ip': '127.0.0.1'}

In [13]:
conn = sqlalchemy.create_engine("mysql+mysqlconnector://{0}:{1}@{2}/".format(creds['username'],
                                                                               creds['password'],
                                                                               creds['ip']), pool_size=10, max_overflow=20).connect()

Run the next cells in order and this will create your database import our `xlsx` pitcher data into the MYSQL instance. 

In [14]:
conn.connect()

In [16]:
conn.execute("CREATE DATABASE Pitchers")

In [17]:
conn.execute("USE Pitchers;")

In [18]:
df.to_sql(con=conn, name='mlb_stats_2018', if_exists='replace', chunksize=1000, index=False)

In [19]:
rs = conn.execute('SELECT * FROM mlb_stats_2018 LIMIT 5;')

for row in rs:
    print(row)

(439, 'Clayton Kershaw', 34000000.0, 30, 'LAD', 'NL', 5, 5, 0.5, 2.58, 17, 17, 0, 0, 0, 0, 101.1, 89, 32, 29, 10, 19, 0, 99, 2, 0, 8, 409, 154.0, 3.1, 1.066, 7.9, 0.9, 1.7, 8.8, 5.21)
(332, 'Zack Greinke', 31954483.0, 34, 'ARI', 'NL', 12, 7, 0.632, 2.89, 24, 24, 0, 0, 0, 0, 149.2, 127, 49, 48, 20, 29, 1, 152, 4, 0, 0, 598, 151.0, 3.52, 1.042, 7.6, 1.2, 1.7, 9.1, 5.24)
(30, 'Jake Arrieta', 30000000.0, 32, 'PHI', 'NL', 9, 6, 0.6, 3.11, 22, 22, 0, 0, 0, 0, 127.1, 113, 61, 44, 11, 40, 0, 91, 4, 0, 9, 520, 134.0, 3.88, 1.202, 8.0, 0.8, 2.8, 6.4, 2.28)
(639, 'David Price', 30000000.0, 32, 'BOS', 'AL', 12, 6, 0.667, 3.75, 23, 23, 0, 1, 0, 0, 134.1, 122, 59, 56, 18, 39, 0, 134, 7, 0, 1, 556, 115.0, 3.92, 1.199, 8.2, 1.2, 2.6, 9.0, 3.44)
(829, 'Justin Verlander', 28000000.0, 35, 'HOU', 'AL', 11, 7, 0.611, 2.5, 25, 25, 0, 1, 1, 0, 158.1, 116, 47, 44, 22, 28, 0, 206, 7, 2, 2, 619, 158.0, 3.02, 0.909, 6.6, 1.3, 1.6, 11.7, 7.36)


If you are seeing data above then your database is filled with the 2018 mlb pitcher data. If you received any errors here check your local mysql server and be sure that it is running and installed correctly on your local machine.

In [20]:
conn.close()